<a href="https://colab.research.google.com/github/jinwu99/Telco-Customer-Churn-Analysis/blob/main/Telco_customer_churn_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [2]:
cd MyDrive/My Drive/Colab Notebooks

/content/MyDrive/My Drive/Colab Notebooks


Data description :
* California 지역, 가상의 통신회사의 약 7000명 3분기 고객데이터
* 출처 : https://www.kaggle.com/datasets/yeanzc/telco-customer-churn-ibm-dataset/

# 분석 목적
* **고객의 서비스이용 패턴 탐색** 및 이탈율이 높은 이용패턴에 대한 **해결방안 제시**

## 분석 과정
1. 고객의 서비스이용 현황 파악
2. 고객의 정보를 바탕으로 군집화
3. 각 고객군집의 특성 파악 및 이탈율 높은 군집 분석

In [3]:
import pandas as pd

Telco = pd.read_csv('Telco_customer_churn.csv')
print(Telco.shape)
Telco.keys()

(7043, 33)


Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Reason'],
      dtype='object')

* 분석에 앞서, 각 범주요인의 이름을 좀 더 구체화하자.
* 또한 Tenure과 Monthly Charges를 3분위수로 범주화하자.

In [4]:
Telco.loc[Telco['Churn Label']=='Yes', 'Churn Label'] = 'Churn_Yes'
Telco.loc[Telco['Churn Label']=='No', 'Churn Label'] = 'Churn_No'
Telco.loc[Telco['Senior Citizen']=='Yes', 'Senior Citizen'] = 'Senior'
Telco.loc[Telco['Senior Citizen']=='No', 'Senior Citizen'] = 'Not_Senior'
Telco.loc[Telco['Partner']=='Yes', 'Partner'] = 'w/Partner'
Telco.loc[Telco['Partner']=='No', 'Partner'] = 'No_Partner'
Telco.loc[Telco['Dependents']=='Yes', 'Dependents'] = 'w/Dependents'
Telco.loc[Telco['Dependents']=='No', 'Dependents'] = 'No_Dependents'
Telco = Telco.drop(columns=['Phone Service'])
Telco.loc[Telco['Multiple Lines']=='Yes', 'Multiple Lines'] = 'Multiple Lines'
Telco.loc[Telco['Multiple Lines']=='No', 'Multiple Lines'] = 'Single Line'
Telco.loc[Telco['Internet Service']=='No', 'Internet Service'] = 'No_Internet_Service'
Telco.loc[Telco['Online Security']=='Yes', 'Online Security'] = 'Online Security'
Telco.loc[Telco['Online Security']=='No', 'Online Security'] = 'No_Online Security'
Telco.loc[Telco['Online Backup']=='Yes', 'Online Backup'] = 'Online Backup'
Telco.loc[Telco['Online Backup']=='No', 'Online Backup'] = 'No_Online Backup'
Telco.loc[Telco['Device Protection']=='Yes', 'Device Protection'] = 'Device Protection'
Telco.loc[Telco['Device Protection']=='No', 'Device Protection'] = 'No_Device Protection'
Telco.loc[Telco['Tech Support']=='Yes', 'Tech Support'] = 'Tech Support'
Telco.loc[Telco['Tech Support']=='No', 'Tech Support'] = 'No_Tech Support'
Telco.loc[Telco['Streaming TV']=='Yes', 'Streaming TV'] = 'Streaming TV'
Telco.loc[Telco['Streaming TV']=='No', 'Streaming TV'] = 'No_Streaming TV'
Telco.loc[Telco['Streaming Movies']=='Yes', 'Streaming Movies'] = 'Streaming Movies'
Telco.loc[Telco['Streaming Movies']=='No', 'Streaming Movies'] = 'No_Streaming Movies'
Telco.loc[Telco['Paperless Billing']=='Yes', 'Paperless Billing'] = 'Paperless Billing'
Telco.loc[Telco['Paperless Billing']=='No', 'Paperless Billing'] = 'No_Paperless Billing'
Telco['Monthly Charges cat'] = pd.qcut(Telco['Monthly Charges'],q=3).cat.codes
Telco.loc[Telco['Monthly Charges cat']==0,'Monthly Charges cat'] = 'charge_low'
Telco.loc[Telco['Monthly Charges cat']==1,'Monthly Charges cat'] = 'charge_mid'
Telco.loc[Telco['Monthly Charges cat']==2,'Monthly Charges cat'] = 'charge_high'
Telco['Tenure Months cat'] = pd.qcut(Telco['Tenure Months'],q=3).cat.codes
Telco.loc[Telco['Tenure Months cat']==0,'Tenure Months cat'] = 'tenure_low'
Telco.loc[Telco['Tenure Months cat']==1,'Tenure Months cat'] = 'tenure_mid'
Telco.loc[Telco['Tenure Months cat']==2,'Tenure Months cat'] = 'tenure_high'

In [5]:
import plotly.io as pio
pio.renderers.default = "colab"

# 1.고객의 서비스이용 현황 파악
고객 데이터는 대략적으로 4가지 정보를 가진다 : \
1. 인구통계학적 정보
2. 이용하는 인터넷/전화 서비스 정보
3. 계약정보
4. 이탈이유

각 정보에 대한 고객의 현황을 살펴보자.

## 1.1. 인구통계학적 정보

In [6]:
import plotly.graph_objects as go

colors = ['lightgreen','salmon']
demo = ['Gender','Senior Citizen','Partner','Dependents']
churn = ['Churn_No','Churn_Yes']
showlegend = [True] + [False]*(len(demo)-1)
spacing = ['',' ','  ']

fig = go.Figure()

for d in range(len(demo)):
    grouped = Telco.groupby([demo[d], 'Churn Label']).size().unstack(fill_value=0).reset_index()
    for c in range(len(churn)):
        fig.add_trace(go.Bar(x=grouped[demo[d]], y=grouped[churn[c]], name=churn[c],
                             marker_color=colors[c], showlegend=showlegend[d]))
    if d < (len(demo)-1):
        fig.add_trace(go.Bar(x=[spacing[d]], y=[0], showlegend=False))

fig.update_layout(
    title="Customer Churn by Demographics",
    yaxis_title="Churn Count",
    legend_title="Churn Label",
    barmode='stack'
)

fig.update_layout(autosize=False,width=1200,height=500)
fig.show()

* 성별 비율이 비슷, 각 성별의 이탈율도 비슷하다.
* Senior(65세 이상)인 경우가 월등히 적되, 이탈율이 더 높은 편.
* 파트너 유무 비율이 비슷, 파트너가 있는 경우의 이탈율이 조금 더 높은 편.
* 부양가족이 있는 경우가 월등히 적되, 이탈율이 더 적은 편.

## 1.2. 이용하는 인터넷/전화 서비스 정보

In [7]:
colors = ['lightgreen','salmon']
service = ['Multiple Lines',
           'Internet Service','Online Security','Online Backup','Device Protection','Tech Support',
           'Streaming TV','Streaming Movies']
churn = ['Churn_No','Churn_Yes']
showlegend = [True] + [False]*(len(service)-1)
spacing = ['',' ','  ','   ','    ','     ','      ','       ','        ']

fig = go.Figure()

for d in range(len(service)):
    grouped = Telco.groupby([service[d], 'Churn Label']).size().unstack(fill_value=0).reset_index()
    if d > 1 :
        grouped = grouped.set_index(service[d]).drop(index='No internet service').reset_index()
    for c in range(len(churn)):
        fig.add_trace(go.Bar(x=grouped[service[d]], y=grouped[churn[c]], name=churn[c],
                             marker_color=colors[c], showlegend=showlegend[d]))
    if d < (len(service)-1):
        fig.add_trace(go.Bar(x=[spacing[d]], y=[0], showlegend=False))

fig.update_layout(
    title="Customer Churn by Service",
    yaxis_title="Churn Count",
    legend_title="Churn Label",
    barmode='stack'
)

fig.update_layout(autosize=False,width=1500,height=500)
fig.show()

* 전화회선은 단일이 제일 많고 이탈고객 수는 다중과 비슷.
* 인터넷회선 중 Fiber optic 이용자가 제일 많으면서 이탈율도 제일 높음.
* 보안, 백업, 보호, tech support 모두 이용자의 이탈율이 비이용자보다 적은 편.
* Streaming service에 대한 차이는 별로 없음.

## 1.3. 계약정보


In [8]:
colors = ['lightgreen','salmon']
service = ['Contract','Paperless Billing','Payment Method','Monthly Charges cat','Tenure Months cat']
churn = ['Churn_No','Churn_Yes']
showlegend = [True] + [False]*(len(service)-1)
spacing = ['',' ','  ','   ','    ','     ','      ','       ','        ']

fig = go.Figure()

for d in range(len(service)):
    grouped = Telco.groupby([service[d], 'Churn Label']).size().unstack(fill_value=0).reset_index()
    for c in range(len(churn)):
        fig.add_trace(go.Bar(x=grouped[service[d]], y=grouped[churn[c]], name=churn[c],
                             marker_color=colors[c], showlegend=showlegend[d]))
    if d < (len(service)-1):
        fig.add_trace(go.Bar(x=[spacing[d]], y=[0], showlegend=False))

fig.update_layout(
    title="Customer Churn by Contract",
    yaxis_title="Churn Count",
    legend_title="Churn Label",
    barmode='stack'
)

fig.update_layout(autosize=False,width=1500,height=500)
fig.show()

* 월계약이 가장 많으면서 이탈율도 가장 높음.
* Paperless Billing이 더 많으면서 이탈율도 더 높음.
* Electronic check가 가장 많으면서 이탈율도 가장 높음.
* 월마다 내는 비용이 높을수록 이탈율도 늘어나는 추세.
* 반면 Tenure은 길어질수록 이탈율이 줄어드는 추세.

## 1.4. 핵심 요약
1. 젊은 사람의 비중이 높지만, 고령자의 이탈율이 더 높다.
2. 부양가족이 있는 경우가 적지만, 이탈율 또한 매우 적다.
3. 인터넷 회선 중 Fiber optic의 이용자 수와 이탈율 모두 제일 높음.
4. 인터넷 부가서비스를 이용하는 경우 이탈율이 더 적음.
5. 계약방법 중 월계약이 가장 많으면서 이탈율도 가장 큼.
6. 월마다 내는 비용이 높을수록 이탈율이 늘어나지만, Tenure가 길어질수록 이탈율이 줄어듬.

## 1.5. 한계점
단일 변수로만 이탈율을 비교하는 것은 다음과 같은 위험성을 가진다 :
1. Simpson's Paradox : A trend appears in several groups of data but disappears or reverses when the groups are combined.
    * 간단한 예로, A그룹의 고객 이탈율이 B그룹의 고객 이탈율보다 높게 나오지만, B그룹의 고객 수가 A그룹보다 월등히 많다면, 두 그룹 모두 합쳤을 때 전반적 이탈율이 작게 나오는 효과를 볼 수 있다. 혹은 그 반대 효과도 나올 수 있다.
        * 우리의 예상보다 고객의 패턴 종류가 다양하다면, Simpson's Paradox가 나올 여지가 더 커질 것이다.
    * 우리의 데이터에도 비슷한 질문들을 적용해볼 수 있다 : 월계약의 이탈율이 높다는 것은, 모든 고객에게 해당되는 이야기일까?
        
2. 변수들 간 상관성 존재 가능성.
    * 특정 인구통계학적 특징을 가진 고객은 특정 서비스들을 연계하여 활용할 가능성이 있다.
        * 어떤 한 요인에서 이탈율이 높게 나와 그 요인에 자칫 손보면, 그 요인과 연관된 다른 요인들도 영향받아 더 안좋은 효과를 낳을 수 있다.
    * 그렇다면 다중변수 분석을 하면 되지 않을까?
        * 2~3개의 변수들로 분석이 가능하겠지만, 그 이상의 갯수로 분석하는 것은 어렵다.

## 1.6. 해결방안
1. 다중변수 분석 용으로 로지스틱 회귀분석 가능.
    * 각 변수에 대한 이탈율을 변수들 간의 상관성을 보정한 상태로 구할 수 있다.
    * 그렇지만 이 역시 Simpson's Paradox 문제를 피하기 어렵다. 고객 그룹마다 특징이 상이할 수 있기 때문.
        * 또한 각 그룹의 특징도 알기 어렵다.
    * 이에 대응하기 위해, 고객 그룹이 어떻게 나뉘는지 알고 있다는 전제하에, 각 그룹에 대해 로지스틱 회귀분석 하거나 모든 고객 데이터에 대해 로지스틱 혼합 모형을 사용해볼 수 있다.
        * 문제는 고객 군집이 어떻게 나뉘는지 알 수 없는 상태이다.

2. 군집분석.
    * 고객의 인구통계학 정보와 서비스사용 등 정보를 바탕으로 군집화하여 그룹으로 나눌 수 있다.
        * 18가지의 범주형 변수에 대해 가능한 고객의 종류는 2,672,922가지이고 현재 고객의 수는 7000여명인 점을 고려하면, 비슷한 특징의 고객들끼리 군집되어있음도 짐작할 수 있다.
    * 다중변수 분석 용으로도 용이할 뿐더러, 앞서 언급하였듯 로지스틱 회귀분석의 연장선으로도 활용 가능하다.

# 2.고객의 정보를 바탕으로 군집화
군집분석은 크게 두 가지로 나눌 수 있다.

1. Distance-based clustering:  말 그대로 데이터 간의 dissimilarity를 먼저 측정하고 클러스터링 방법을 적용시키는 방법들을 말한다.
    * 다음 4가지를 결정해야 한다.
        1. 어떤 변수들을 클러스터링에 활용할 것인가?
        2. 어떤 distance metric을 활용할 것인가?
        3. 어떤 철학,Criterion의 클러스터링 방법을 활용할 것인가?
    * 인구통계학 정보와 서비스이용 정보, 그리고 계약정보 모두 활용한다면 총 18가지의 범주형 변수를 고려하게 된다. 또한 범주형인 점을 고려하여 데이터간의 거리를 재기 위해 Simple matching이나 Gower distance 등을 활용할 수 있을 것이다.
        * 이에 따라 K-modes/median, hierarchical clustering 등 다양하게 적용할 수 있다는 점에서 꽤나 flexible하다.
    * 클러스터링 이후, 각 군집의 특징을 설명하기 위해 변수들과의 연관성도 측정해야한다. 클러스터 라벨도 범주형 변수로 취급한다면, Chi-square distance나 Simple matching 등 이 또한 다양한 방법들을 쓸 수 있을 것이다.
    <!-- * 그러나 거리 기반 클러스터링은 차원의 저주 문제가 있을 수 있다는 것. -->
        <!-- * 일차원에서 거리가 2cm였던 물체가 10차원에서는 거리가 무려 약 1km가 되어버리는 현상으로 볼 수 있다.  -->
        <!-- * 사실 Distance metric을 어떤 것 쓰냐 따라 차원의 저주 문제에 덜 민감할 수 있고, 또한 텍스트나 이미지와 같이 low dimension space에 워낙에 clustered되있는 데이터의 경우에도 덜 민감할 수 있다. 하지만 어디까지나 그럴 수 있다는 가능성의 얘기이다. -->


2. Model-based clustering: 데이터가 어떤 분포/방정식으로부터 생성되었을 것이라는 가정을 가지는 클러스터링 방법들을 말한다.
    * (항상은 아니지만) 주로 해석력에서 우수한 특징을 가지는데, 변수들 간의 관계식을 직관적 논리 아래 가정하기 때문이다.
    * 범주형 변수들을 바탕으로 군집화하는 대표적인 방법은 Latent Class Analysis(LCA)로써, 다음과 같은 분포 가정을 가진다.
    \begin{align}
        (Z_1,...,Z_K) &\sim Multinomial(\phi_1,...,\phi_K), & \phi_k = P(Z_k=1) \\
        (X_1,...,X_J)|Z_k=1 &\sim Multinomial(\theta_{k,1},...,\theta_{k,J}), & \theta_{k,j} = P(X_j|Z_k=1) \\
        P(X_1,...,X_J) &= \sum_{k=1}^K P(Z_k=1)P(X_1,...,X_J|Z_k=1) \\
        &= \sum_{k=1}^K P(Z_k=1) \prod_{j=1}^J P(X_j|Z_k=1)
    \end{align}
    * 각 범주요인이 클러스터에 속할 확률을 가진다는 점에서 해석이 용이해진다 - 어떤 고객 군집이 확률적으로 어떤 서비스들을 많이 사용하는 편인지 등의 해석이 가능한 것이다.

* Distance-based clustering(DBC) versus LCA
    * DBC는 LCA보다 다양한 방법으로 클러스터링할 수 있지만, 최선의 방법을 찾기 어려울 수 있다.
        * DBC는 Distance metric과 Clustering criterion에 의존하는 방법론이지, 해석력을 높이기 위한 방법론은 아니다.
    * 반면 LCA는 해석적인데다 거리기반 클러스터링보다 차원의 저주 문제에 자유로운 편(LCA의 알고리즘에 데이터간 거리측정하는 식이 없다).
    * 우리의 목표는 군집화와 더불어 각 군집의 특징도 해석하는 것이기 때문에, LCA를 사용하기로 택한다.
    

In [9]:
df_cat = Telco[['Gender', 'Senior Citizen', 'Partner', 'Dependents',
                'Multiple Lines', 'Internet Service',
                'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
                'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
                'Payment Method', 'Monthly Charges cat','Tenure Months cat', 'Churn Label']]

In [10]:
%%capture
!pip install stepmix

In [11]:
%%capture
from stepmix.stepmix import StepMix
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

category_mappings = {}
for col in df_cat.columns:
    label_encoder = LabelEncoder()
    df_cat[col] =  label_encoder.fit_transform(df_cat[col])
    category_mappings[col] = dict(zip(label_encoder.classes_,
                                      label_encoder.transform(label_encoder.classes_)))

## 2.1. 최적의 클러스터 갯수
* BIC와 같은 지표로 최적의 클러스터 갯수를 정할 수 있다.
* 하지만 LCA 추정하는 EM 알고리즘은 클러스터 갯수를 많이 설정할수록 local optima에 취약하며, 클러스터 갯수가 많을 수록 최적이라고 진단하는 경향성도 가진다.
    * 따라서 고려하려는 클러스터 갯수 범위를 어느 정도 제한하는 것이 좋다.
* 최소한 하나의 클러스터에 700명은 존재한다는 가정을 한다면 - 클러스터 갯수 후보로 3개부터 9개까지 고려할 수 있다.

In [12]:
import plotly.express as px

k_candidates = range(3,10,1)
results = dict(param_n_components=[], bic=[])
for k in k_candidates:
    model = StepMix(n_components=k, measurement="categorical", random_state=99)
    model.fit(df_cat)
    results['param_n_components'].append(k)
    results['bic'].append(model.bic(df_cat))

fig = px.line(pd.DataFrame(results), x="param_n_components", y="bic")

fig.update_layout(
    title="Model Selection via BIC",
    yaxis_title="BIC",
    xaxis_title="Number of clusters",
)

fig.update_layout(autosize=False,width=600,height=400)
fig.show()

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:01<00:00,  1.71s/it, max_LL=-8.48e+4, max_avg_LL=-12]


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:06<00:00,  6.26s/it, max_LL=-8.2e+4, max_avg_LL=-11.6]


Fitting StepMix...


...

Initializations (n_init) : 100%|██████████| 1/1 [00:10<00:00, 10.34s/it, max_LL=-7.83e+4, max_avg_LL=-11.1]


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:06<00:00,  6.46s/it, max_LL=-7.79e+4, max_avg_LL=-11.1]


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:24<00:00, 24.92s/it, max_LL=-7.78e+4, max_avg_LL=-11.1]


* BIC는 낮을 수록 좋으며, 8개가 최적의 클러스터 갯수가 된다. 이제 각 클러스터의 고객수와 Churn rate을 확인해보자.

In [13]:
%%capture
num_cluster = 8
model = StepMix(n_components=num_cluster, measurement="categorical", random_state=99)
model.fit(df_cat)
df_cat['cluster'] = model.predict(df_cat)

# 3. 각 고객군집의 특성 파악 및 이탈율 높은 군집 분석
* 각 클러스터의 고객 수와 Churn rate을 확인해보자.

In [14]:
import plotly.express as px

cluster_counts = pd.DataFrame(df_cat['cluster'].value_counts().reset_index()).rename(columns={'index':'cluster', 'cluster':'counts'})
cluster_churn_counts = df_cat.groupby(['cluster','Churn Label']).size().unstack()
cluster_churn_rate = (cluster_churn_counts[1]/cluster_churn_counts.sum(axis=1)).reset_index().rename(columns={0:'Churn rate'})

cluster_churn_counts = cluster_churn_counts.reset_index().rename(columns={0:'No Churn', 1:'Churn'})
cluster_rename = {}
for k in range(num_cluster):
    cluster_rename[k] = 'Cluster ' + str(k+1)

cluster_churn_counts = cluster_churn_counts.reset_index().rename(columns={0:'No Churn', 1:'Churn'})
cluster_churn_counts['cluster'] = cluster_churn_counts.cluster.map(cluster_rename)

fig = px.bar(cluster_churn_counts, x="cluster", y=["No Churn", "Churn"])

fig.update_layout(
    title="Cluster counts and Churn ratio",
    yaxis_title="Count",
    xaxis_title="Clusters",
)

fig.update_layout(autosize=False,width=1000,height=500)
fig.show()

# cluster_churn = pd.merge(cluster_counts,cluster_churn_rate,left_on='cluster',right_on='cluster')
# cluster_churn['cluster'] += 1
# cluster_churn

* 각 클러스터의 고객수는 약 700명에서 1300명의 범위를 가진다.
* cluster 3, 7에서 이탈율이 50%를 넘긴다.
* 각 클러스터의 특징을 위해 LCA 결과를 radar chart으로 확인해보자.

In [15]:
mm = model.get_mm_df().round(2)
mm.index = [levels[2] for levels in mm.index]

new_cat_map = {}
for col in df_cat:
    if col=='cluster' : continue
    for idx, (key, val) in enumerate(category_mappings[col].items()) :
        new_cat_map[col + '_' + str(val)] = key

mm = mm.rename(index=new_cat_map)

In [16]:
# 각 범주요인의 글자 길이를 줄이기 위함.
mm.index = ['No Churn','Churn','Monthly','1yr','2yr',
            'No Dep','Dependents','Protect',
            'No int','No protect','Female','Male','DSL',
            'Fiber optic','No int','High fee','Low fee',
            'Mid fee','Multiple','No phone','Single',
            'No int','No onl bck','Backup',
            'No int','No onl sec','Security',
            'No ppb','PaperX Bill','No_part','Partner',
            'Bank','Credit card',
            'Electronic','Mailed','No senior','Senior',
            'No int','No stream mv','Stream MV',
            'No int','No stream tv','Stream TV',
            'No int','No tech','Tech Sup','High tenure',
            'Low tenure','Mid tenure']

In [17]:
# State of the contract
# state_label = ['Churn_Yes','Month-to-month','One year','Two year',
#                'charge_high','charge_mid','charge_low',
#                'tenure_high','tenure_mid','tenure_low']
state_label = ['Churn','Monthly','1yr','2yr',
               'High fee','Mid fee','Low fee',
               'High tenure','Mid tenure','Low tenure']
# Demographics
# demo_label = ['w/Dependents','w/Partner','Female','Male','Senior']
demo_label = ['Dependents','Partner','Female','Male','Senior']
# Billing method
# bill_label = ['Paperless Billing','Bank transfer (automatic)','Credit card (automatic)',
#               'Electronic check','Mailed check']
bill_label = ['PaperX Bill','Bank','Credit card',
              'Electronic','Mailed']
# Use Additiona; service?
# service_label = ['Multiple Lines','Single Line','DSL','Fiber optic',
#                  'Device Protection','Online Backup','Online Security',
#                  'Streaming Movies','Streaming TV','Tech Support']
service_label = ['Multiple','Single','DSL','Fiber optic',
                 'Protect','Backup','Security',
                 'Stream MV','Stream TV','Tech Sup']

label_list = [state_label,demo_label,bill_label,service_label]
id_vars_list = ['Contract_Status','Demographics','Billing','Service']

rename_col_list = []
for j in range(len(label_list)):
    rename_col = {}
    rename_col['index'] = id_vars_list[j]
    for k in range(num_cluster):
        rename_col[k] = 'cluster_' + str(k+1)
    rename_col_list.append(rename_col)

lca_list = [mm.loc[labels,:].reset_index() for labels in label_list]
lca_list = [lca_list[j].rename(columns=rename_col_list[j]) for j in range(len(label_list))]
lca_long_list = [lca_list[j].melt(id_vars=id_vars_list[j], var_name='Cluster', value_name='Probability') for j in range(len(label_list))]

In [18]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def Radar_SubPlot_for_Clusters(j,fig,row,col,show):
    colors = px.colors.qualitative.Set2
    for k in range(num_cluster):
        r = lca_long_list[j].loc[lca_long_list[j]['Cluster']=='cluster_'+str(k+1), 'Probability'].tolist()
        theta = lca_long_list[j].loc[lca_long_list[j]['Cluster']=='cluster_'+str(k+1), id_vars_list[j]].tolist()
        fig.add_trace(go.Scatterpolar(
            r=r,
            theta=theta,
            fill='toself',
            line_color=colors[k],
            opacity=0.8,
            name='Cluster '+str(k+1),
            showlegend=show,
        ),row=row, col=col)
        if row==8:
            row=1; col+=1
        else:
            row+=1
    return(fig,col)

def Radar_Plot_for_Clusters():
    fig = make_subplots(rows=8, cols=4,
                        specs=[[{'type': 'polar'}]*4]*8)
    show_legend = [True] + [False]*3
    annotations_list = [
       go.layout.Annotation(
            showarrow=False,
            text='<b>Contract</b>',
            x=0.06,
            y=1.04,
            font=dict(
                size=18
            )
        ),
        go.layout.Annotation(
            showarrow=False,
            text='<b>Demographics</b>',
            x=0.3,
            y=1.04,
            font=dict(
                size=18
            )
        ),
        go.layout.Annotation(
            showarrow=False,
            text='<b>Pay & Bill</b>',
            x=0.63,
            y=1.04,
            font=dict(
                size=18
            )
        ),
        go.layout.Annotation(
            showarrow=False,
            text='<b>Service</b>',
            x=0.93,
            y=1.04,
            font=dict(
                size=18
            )
        )]
    row = col = 1
    for j in range(4):
        fig, col = Radar_SubPlot_for_Clusters(j,fig,row,col,show_legend[j])
    fig.update_polars(radialaxis=dict(range=[0, 1]))
    fig.update_layout(annotations=annotations_list)
    return(fig)

In [19]:
fig = Radar_Plot_for_Clusters()


fig.update_layout(autosize=False,width=1300,height=1500)
fig.show()

* Radar모양으로 클러스터간 구분이 쉽고 특징도 알기 쉽다.
* 하지만 클러스터가 많은 만큼 여전히 한눈에 파악하기 어렵다. Heatmap으로 다시 나타내보자.

In [20]:
import numpy as np

reorder_cluster = [5,1,2,6,3,7,4,0]
mm = mm[reorder_cluster]

label_list = [['Churn'],
              ['Multiple','Single'],
              ['DSL','Fiber optic'],
              ['Protect'],
              ['Backup'],
              ['Security'],
              ['Stream MV'],
              ['Stream TV'],
              ['Tech Sup'],
              ['High fee','Mid fee','Low fee'],
              ['High tenure','Mid tenure','Low tenure'],
              ['Monthly','1yr','2yr'],
              ['Female','Male'],
              ['Dependents'],
              ['Partner'],
              ['Senior'],
              ['Bank','Credit card','Electronic','Mailed'],
              ['PaperX Bill']]
x_axis = ['Churn','Phone','Internet','Protect','Backup','Security','Stream MV','Stream TV','Tech Sup','Charge','Tenure','Contract','Gender','Depend','Partner','Senior','Payment','PaperX Bill']
x_axis = ['<b>'+x+'</b>' for x in x_axis]
df_list = [mm.loc[label,:] for label in label_list]
df_list = [df.rename(columns=rename_col_list[0]) for df in df_list]

most_cat_list = np.array([[[label_list[j][np.argmax(df_list[j][col])]] for col in df_list[j].columns] for j in range(len(df_list))])
most_prob_list = np.array([[[df_list[j][col][np.argmax(df_list[j][col])]] for col in df_list[j].columns] for j in range(len(df_list))])

show_threshold = [1/2] + [1/3] + [1/2]*7 + [1/3]*3 + [1/2]*4 + [1/4] + [1/2]
for j in range(len(df_list)): most_cat_list[j][most_prob_list[j] < show_threshold[j]] = ''

most_cat = most_cat_list[0]
most_prob = most_prob_list[0]
for i in range(1,len(most_cat_list),1):
    most_cat = np.concatenate([most_cat,most_cat_list[i]],axis=1).tolist()
    most_prob = np.concatenate([most_prob,most_prob_list[i]],axis=1).tolist()

In [21]:
import plotly.graph_objects as go

cluster_name = ['<b>'+'Cluster '+str(k+1)+'</b>' for k in reorder_cluster]

fig = go.Figure(data=go.Heatmap(
    x=x_axis,
    y=cluster_name,
    z=most_prob,
    text=most_cat,
    texttemplate="%{text}",
    colorscale='RdBu',
    textfont={"size":13,"family":'Ariel'},
    zmin=0, zmax=1
))

fig.update_layout(
    title='Most Probable Category Heatmap',
    xaxis_title='',
    yaxis_title='',
)

fig.update_xaxes(side="top",tickfont_size=13,tickfont_family='Ariel')
fig.update_yaxes(tickfont_size=13,tickfont_family='Ariel')
fig.update_layout(autosize=False,width=1500,height=500)
fig.show()

* 각 범주형 변수에서 확률이 가장 큰 범주요인만 색상와 글씨를 표시하였다.
    * 예를 들어, Payment에서 Electronic의 확률이 0.3으로 가장 크다면 그것만 표시하였다.
* 만약 범주요인 중 'No..'라는 글자를 가진 경우 시각화에 제외하였다.

위의 8개의 클러스터를 대략 4개로도 묶어 볼 수 있다.
1. Cluster 1, Cluster 5 : Single 전화서비스만 이용하는 고객군.
    * Cluster 5가 tenure이 더 길고 계약기간도 길며, Partner가 존재하는 경향이 더 크다.
2. Cluster 8, Cluster 4 : Single 전화서비스와 DSL 인터넷을 월계약으로 이용하는 고객군.
    * Cluster 4가 Tenure이 더 길고  Single 전화서비스 사용 경향이 더 크다.
3. Cluster 7, Cluster 3 : Fiber optic 인터넷을 월계약으로 이용하는데 이탈율이 50% 이상인 위험고객군.
    * Cluster 7은 Multiple 전화서비스에다 Streaming 서비스도 이용하는 고소비 고객층.
4. Cluster 2, Cluster 6 : 인터넷 부가서비스 두루두루 이용하면서 긴 계약기간에 tenure도 가장 긴 VIP고객군.
    * Cluster 2가 상대적으로 전화서비스 이용안하고 DSL 인터넷 사용하기 때문에, Cluster 6로 유입시키는 전략을 짜야한다.

In [22]:
# df_cat_reg = df_cat
# df_cat_reg.to_csv('Telco_clustered.csv',index=False)
#df_cat_reg = pd.read_csv('Telco_clustered.csv')
#df_cat_reg.columns

In [23]:
df_cat_reg = pd.read_csv('df_cat_reg.csv')

In [24]:
df_cat_reg_list = [df_cat_reg[df_cat_reg['cluster']==k].reset_index(drop=True) for k in range(num_cluster)]
for k in range(num_cluster):
    df_cat_reg_list[k].columns = df_cat_reg_list[k].columns.str.replace(" ","_")

In [25]:
cluster_consider = [2,6,7]

model_formula = "Churn_Label ~ "
model_formula_list = [model_formula for k in cluster_consider]
covariate_drop_list = []
covariate_drop_list.append(['Internet_Service','Dependents','Monthly Charges cat'])
covariate_drop_list.append(['Internet_Service','Online_Security','Streaming_TV','Streaming_Movies','Tech_Support','Dependents','Monthly Charges cat'])
covariate_drop_list.append(['Internet_Service','Monthly Charges cat'])

for j in range(len(cluster_consider)):
    for col in df_cat_reg_list[cluster_consider[j]].columns:
        if col not in covariate_drop_list[j]+['Churn_Label','cluster','Contract']:
            model_formula_list[j] += 'C(' + col + ')'
            if col != 'Tenure_Months_cat':
                model_formula_list[j] += ' + '

In [26]:
import statsmodels.formula.api as smf

model_list = []
for j in range(len(cluster_consider)):
    model_list.append(smf.logit(model_formula_list[j], data=df_cat_reg_list[cluster_consider[j]]).fit())

Optimization terminated successfully.
         Current function value: 0.627337
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.629740
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.569314
         Iterations 6


In [27]:
model_list[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_Label   No. Observations:                 1296
Model:                          Logit   Df Residuals:                     1279
Method:                           MLE   Df Model:                           16
Date:                Mon, 06 Nov 2023   Pseudo R-squ.:                 0.08790
Time:                        05:51:31   Log-Likelihood:                -813.03
converged:                       True   LL-Null:                       -891.38
Covariance Type:            nonrobust   LLR p-value:                 3.705e-25
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -0.9246      0.326     -2.835      0.005      -1.564      -0.285
C(Gender)[T.1]               -0.2332      0.119     -1.952      0.051      -0.467       0.001
C(Senior_Citizen)[T.1]        0.1336      0.131      1.020      0.308      -0.123       0.390
C(Partner)[T.1]               0.1154      0.124      0.932      0.351      -0.127       0.358
C(Multiple_Lines)[T.2]       -0.5561      0.137     -4.050      0.000      -0.825      -0.287
C(Online_Security)[T.2]      -0.2008      0.155     -1.294      0.196      -0.505       0.103
C(Online_Backup)[T.2]        -0.0640      0.125     -0.514      0.607      -0.308       0.180
C(Device_Protection)[T.2]    -0.0272      0.121     -0.224      0.823      -0.265       0.210
C(Tech_Support)[T.2]         -0.1969      0.147     -1.341      0.180      -0.485       0.091
C(Streaming_TV)[T.2]          0.4511      0.140      3.233      0.001       0.178       0.724
C(Streaming_Movies)[T.2]      0.4066      0.139      2.921      0.003       0.134       0.679
C(Paperless_Billing)[T.1]     0.3697      0.155      2.378      0.017       0.065       0.674
C(Payment_Method)[T.1]       -0.0624      0.221     -0.282      0.778      -0.496       0.372
C(Payment_Method)[T.2]        0.1327      0.169      0.786      0.432      -0.198       0.463
C(Payment_Method)[T.3]        0.0266      0.273      0.097      0.922      -0.509       0.562
C(Tenure_Months_cat)[T.1]     1.3608      0.208      6.551      0.000       0.954       1.768
C(Tenure_Months_cat)[T.2]     0.0489      0.174      0.281      0.779      -0.293       0.390
=============================================================================================
"""

유의한 변수 : Multiple Lines, Streaming_TV, Streaming_Movies, Paperless_Billing, Tenure_middle

In [28]:
model_list[1].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_Label   No. Observations:                  894
Model:                          Logit   Df Residuals:                      881
Method:                           MLE   Df Model:                           12
Date:                Mon, 06 Nov 2023   Pseudo R-squ.:                 0.07150
Time:                        05:51:31   Log-Likelihood:                -562.99
converged:                       True   LL-Null:                       -606.34
Covariance Type:            nonrobust   LLR p-value:                 2.132e-13
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -1.7609      0.469     -3.756      0.000      -2.680      -0.842
C(Gender)[T.1]               -0.0425      0.144     -0.295      0.768      -0.325       0.240
C(Senior_Citizen)[T.1]        0.0390      0.162      0.241      0.809      -0.278       0.356
C(Partner)[T.1]               0.0859      0.163      0.528      0.598      -0.233       0.405
C(Multiple_Lines)[T.2]       -0.1507      0.153     -0.985      0.325      -0.451       0.149
C(Online_Backup)[T.2]         0.0431      0.191      0.225      0.822      -0.332       0.418
C(Device_Protection)[T.2]     0.0565      0.215      0.263      0.793      -0.365       0.478
C(Paperless_Billing)[T.1]     0.3754      0.168      2.241      0.025       0.047       0.704
C(Payment_Method)[T.1]       -0.4218      0.278     -1.515      0.130      -0.967       0.124
C(Payment_Method)[T.2]        0.2637      0.217      1.217      0.224      -0.161       0.688
C(Payment_Method)[T.3]       -0.2030      0.278     -0.731      0.465      -0.747       0.341
C(Tenure_Months_cat)[T.1]     2.1308      0.384      5.553      0.000       1.379       2.883
C(Tenure_Months_cat)[T.2]     1.2471      0.384      3.247      0.001       0.494       2.000
=============================================================================================
"""

유의한 변수 : Paperless_Billing(+), Tenure_Middle(+), Tenure_High(+)

In [29]:
model_list[2].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_Label   No. Observations:                  698
Model:                          Logit   Df Residuals:                      679
Method:                           MLE   Df Model:                           18
Date:                Mon, 06 Nov 2023   Pseudo R-squ.:                  0.1358
Time:                        05:51:31   Log-Likelihood:                -397.38
converged:                       True   LL-Null:                       -459.81
Covariance Type:            nonrobust   LLR p-value:                 5.041e-18
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -2.4538      0.770     -3.188      0.001      -3.962      -0.945
C(Gender)[T.1]                0.2908      0.176      1.652      0.098      -0.054       0.636
C(Senior_Citizen)[T.1]        0.6180      0.243      2.543      0.011       0.142       1.094
C(Partner)[T.1]               0.1470      0.209      0.702      0.483      -0.264       0.558
C(Dependents)[T.1]           -0.9634      0.273     -3.527      0.000      -1.499      -0.428
C(Multiple_Lines)[T.1]        0.3986      0.490      0.813      0.416      -0.563       1.360
C(Multiple_Lines)[T.2]        0.0609      0.475      0.128      0.898      -0.870       0.992
C(Online_Security)[T.2]      -0.1968      0.239     -0.822      0.411      -0.666       0.273
C(Online_Backup)[T.2]        -0.3406      0.236     -1.441      0.149      -0.804       0.123
C(Device_Protection)[T.2]    -0.1428      0.261     -0.548      0.584      -0.654       0.368
C(Tech_Support)[T.2]         -0.6708      0.257     -2.606      0.009      -1.175      -0.166
C(Streaming_TV)[T.2]          0.2739      0.298      0.920      0.358      -0.310       0.857
C(Streaming_Movies)[T.2]      0.1552      0.285      0.544      0.587      -0.404       0.715
C(Paperless_Billing)[T.1]     0.5520      0.183      3.009      0.003       0.192       0.911
C(Payment_Method)[T.1]       -0.1515      0.346     -0.438      0.662      -0.830       0.527
C(Payment_Method)[T.2]        0.3404      0.280      1.216      0.224      -0.208       0.889
C(Payment_Method)[T.3]       -0.2220      0.291     -0.763      0.445      -0.792       0.348
C(Tenure_Months_cat)[T.1]     1.8394      0.482      3.815      0.000       0.894       2.784
C(Tenure_Months_cat)[T.2]     0.5023      0.485      1.035      0.301      -0.449       1.453
=============================================================================================
"""

유의한 변수 : Senior(+), Dependents(-), Tech Support(-), Paperless_Billing(+), Tenure_Middle(+)

In [30]:
model_formula = "Churn_Label ~ "
model_formula_list = [model_formula for k in cluster_consider]
reduced_covariate_list = []
reduced_covariate_list.append(['Multiple_Lines','Streaming_TV','Streaming_Movies','Paperless_Billing','Tenure_Months_cat'])
reduced_covariate_list.append(['Paperless_Billing','Tenure_Months_cat'])
reduced_covariate_list.append(['Senior_Citizen','Dependents','Tech_Support','Paperless_Billing','Tenure_Months_cat'])

for j in range(len(cluster_consider)):
    for col in df_cat_reg_list[cluster_consider[j]].columns:
        if col in reduced_covariate_list[j]:
            model_formula_list[j] += 'C(' + col + ')'
            if col != 'Tenure_Months_cat':
                model_formula_list[j] += ' + '

reduced_model_list = []
for j in range(len(cluster_consider)):
    reduced_model_list.append(smf.logit(model_formula_list[j], data=df_cat_reg_list[cluster_consider[j]]).fit())

model_bic = [model_list[j].bic for j in range(3)]
reduced_model_bic = [reduced_model_list[j].bic for j in range(3)]
print(model_bic)
print(reduced_model_bic)

Optimization terminated successfully.
         Current function value: 0.631959
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.637557
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.586210
         Iterations 6
[1747.8964078253375, 1214.319039929035, 919.1782717199117]
[1688.2068173290934, 1167.1355805615317, 864.1860813382881]


In [31]:
reduced_model_list[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_Label   No. Observations:                 1296
Model:                          Logit   Df Residuals:                     1289
Method:                           MLE   Df Model:                            6
Date:                Mon, 06 Nov 2023   Pseudo R-squ.:                 0.08118
Time:                        05:51:31   Log-Likelihood:                -819.02
converged:                       True   LL-Null:                       -891.38
Covariance Type:            nonrobust   LLR p-value:                 1.010e-28
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -1.0560      0.244     -4.325      0.000      -1.535      -0.577
C(Multiple_Lines)[T.2]       -0.6168      0.134     -4.588      0.000      -0.880      -0.353
C(Streaming_TV)[T.2]          0.4971      0.136      3.646      0.000       0.230       0.764
C(Streaming_Movies)[T.2]      0.4601      0.135      3.401      0.001       0.195       0.725
C(Paperless_Billing)[T.1]     0.3919      0.154      2.552      0.011       0.091       0.693
C(Tenure_Months_cat)[T.1]     1.3677      0.194      7.036      0.000       0.987       1.749
C(Tenure_Months_cat)[T.2]     0.0218      0.170      0.128      0.898      -0.312       0.355
=============================================================================================
"""

- Multiple 전화서비스는 이탈율 감소와 연관성이 있다.
- Streaming 서비스와 Paperless Billing은 이탈율 증가와 연관성이 있다.
- Tenure가 low인 경우보다 middle일때 이탈율 증가와 연관성이 크다.
    - Tenure가 high인 경우 유의하지 않다. Tenure가 high가 되기 이전까지 이탈하는 경향성이 있다는것.

In [32]:
reduced_model_list[1].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_Label   No. Observations:                  894
Model:                          Logit   Df Residuals:                      890
Method:                           MLE   Df Model:                            3
Date:                Mon, 06 Nov 2023   Pseudo R-squ.:                 0.05998
Time:                        05:51:32   Log-Likelihood:                -569.98
converged:                       True   LL-Null:                       -606.34
Covariance Type:            nonrobust   LLR p-value:                 1.109e-15
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -1.7063      0.378     -4.512      0.000      -2.447      -0.965
C(Paperless_Billing)[T.1]     0.4091      0.164      2.490      0.013       0.087       0.731
C(Tenure_Months_cat)[T.1]     2.1116      0.365      5.785      0.000       1.396       2.827
C(Tenure_Months_cat)[T.2]     1.2110      0.377      3.210      0.001       0.472       1.950
=============================================================================================
"""

* Paperless_Bill은 이탈율 증가와 연관성이 있다.
* Tenure이 middle인 경우와 high인 경우 모두 이탈율 증가와 연관성이 있다.

In [33]:
reduced_model_list[2].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_Label   No. Observations:                  698
Model:                          Logit   Df Residuals:                      691
Method:                           MLE   Df Model:                            6
Date:                Mon, 06 Nov 2023   Pseudo R-squ.:                  0.1101
Time:                        05:51:32   Log-Likelihood:                -409.17
converged:                       True   LL-Null:                       -459.81
Covariance Type:            nonrobust   LLR p-value:                 1.359e-19
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -1.9882      0.453     -4.392      0.000      -2.875      -1.101
C(Senior_Citizen)[T.1]        0.8314      0.233      3.575      0.000       0.376       1.287
C(Dependents)[T.1]           -0.9478      0.253     -3.749      0.000      -1.443      -0.452
C(Tech_Support)[T.2]         -0.6436      0.245     -2.625      0.009      -1.124      -0.163
C(Paperless_Billing)[T.1]     0.6534      0.177      3.701      0.000       0.307       0.999
C(Tenure_Months_cat)[T.1]     1.5649      0.433      3.612      0.000       0.716       2.414
C(Tenure_Months_cat)[T.2]     0.3405      0.454      0.750      0.454      -0.550       1.231
=============================================================================================
"""

- Senior인 경우 이탈율 증가와 연관성이 있다.
    - 반면, 부양자가 있는 경우 이탈율 감소와 연관성이 있다.
- Tech Support 서비스는 이탈율 감소와 연관성이 있다.
- Paperless Billing은 이탈율 증가와 연관성이 있다.
- Tenure가 low인 경우보다 middle일때 이탈율 증가와 연관성이 크다.
    - Tenure가 high인 경우 유의하지 않다. Tenure가 high가 되기 이전까지 이탈하는 경향성이 있다는것.